In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.streaming import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import os

spark = SparkSession.builder.master("local").appName("DFEmpty").getOrCreate()
sc = spark.sparkContext

In [13]:
# sql adaptive query execution adaptive.coalescePartitions.enabled will makr paritions dynamic
sc.setLogLevel("Error")
spark.conf.set("spark.sql.shuffle.partitions",3)
spark.conf.get("spark.sql.shuffle.partitions")
spark.conf.set("spark.sql.adaptive.enabled","false")
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled","false")
spark.conf.set("spark.sql.execution.arrow.enabled",True)
spark.conf.set("spark.sql.execution.arrow.fallback.enabled",True)

"""
DF is empty ?
when the dataset is empty, it will take huge time for count and first operations
df.rdd.Isempty() is more appropriate
"""

#WithoutInferSchema
#headerTrue will read first row and assign column names but type is String for all
#spark job created to read first column
filepath = "file:///C:/Users/venka/PycharmProjects/pythonProject/dataset/"
df = spark.read.option("header",True) \
            .option("delimiter","|") \
            .option("inferSchema",True) \
            .csv(filepath + "IntUspopulation.csv")

In [14]:
df.show()

+---------+----------------+----------+-------------+-----------+------------------+
|2019_rank|            City|State_Code|2019_estimate|2010_Census|            Change|
+---------+----------------+----------+-------------+-----------+------------------+
|        1|     New York[d]|        NY|      8336817|    8175133|            0.0198|
|        2|     Los Angeles|        CA|      3979576|    3792621|            0.0493|
|        3|         Chicago|        IL|      2693976|    2695598|−0.06%            |
|        4|      Houston[3]|        TX|      2320268|    2100263|            0.1048|
|        5|         Phoenix|        AZ|      1680992|    1445632|            0.1628|
|        6|     San Antonio|        TX|      1547253|    1327407|            0.1656|
|        7|       San Diego|        CA|      1423851|    1307402|            0.0891|
|        8|          Dallas|        TX|      1343573|    1197816|            0.1217|
|        9|        San Jose|        CA|      1021795|     945942|

In [15]:
df1 = df.filter("State_code = 'NY'").groupBy("city").sum("2019_estimate")

In [16]:
df1.show()

+-----------+------------------+
|       city|sum(2019_estimate)|
+-----------+------------------+
|   Syracuse|            142327|
|  Rochester|            205695|
|New York[d]|           8336817|
|    Buffalo|            255284|
|    Yonkers|            200370|
+-----------+------------------+



In [21]:
#Method1
if df1.count() > 0:
    print ("true") 
else:
    print("false")

true


In [24]:
#Method2
if df1.take(1):
    print ("true") 
else:
    print("false")

true


In [26]:
#Method3
if df1.first():
    print ("true") 
else:
    print("false")

true


In [27]:
#Method4
if df1.rdd.isEmpty() :
    print ("false") 
else:
    print("true")

true
